In [1]:
import numpy as np
import pandas as pd
# from tqdm import tqdm
# from scipy.sparse import coo_matrix
from pyspark.ml.recommendation import ALS

# import sklearn
# import random 

from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, VectorAssembler, IndexToString
from pyspark.ml.feature import QuantileDiscretizer
from pyspark.ml.evaluation import MulticlassClassificationEvaluator



In [2]:
from pyspark.sql import SparkSession


MAX_MEMORY = "5g"
spark = SparkSession.builder.appName("recommender")\
.config("spark.executor.memory", MAX_MEMORY)\
.config("spark.driver.memory", MAX_MEMORY)\
.getOrCreate()

# spark = SparkSession\
#         .builder\
#         .appName('recommender_system')\
#         .getOrCreate()

22/05/02 19:42:18 WARN Utils: Your hostname, RedStone-MacBookAir.local resolves to a loopback address: 127.0.0.1; using 172.30.1.45 instead (on interface en0)
22/05/02 19:42:18 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/05/02 19:42:18 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
# 데이터 불러오기
customers = pd.read_csv("data/customers.csv")
tran = pd.read_csv("data/transactions_train.csv")
item = pd.read_csv("data/articles.csv")

In [15]:
customers.head(3)

,customer_id,FN,Active,club_member_status,fashion_news_frequency,age,postal_code
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,NaN,NaN,ACTIVE,NONE,49.0,52043ee2162cf5aa7ee79974281641c6f11a68d276429a...
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,NaN,NaN,ACTIVE,NONE,25.0,2973abc54daa8a5f8ccfe9362140c63247c5eee03f1d93...
2,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,NaN,NaN,ACTIVE,NONE,24.0,64f17e6a330a85798e4998f62d0930d14db8db1c054af6...


In [4]:
tran['count'] = '1'


In [5]:
ALL_USERS = customers['customer_id'].unique().tolist()
ALL_ITEMS = item['article_id'].unique().tolist()

user_ids = dict(list(enumerate(ALL_USERS)))
item_ids = dict(list(enumerate(ALL_ITEMS)))

user_map = {u: uidx for uidx, u in user_ids.items()}
item_map = {i: iidx for iidx, i in item_ids.items()}

tran['user_id'] = tran['customer_id'].map(user_map)
tran['item_id'] = tran['article_id'].map(item_map)

# del customers, customers


In [18]:
tran.head()

,t_dat,customer_id,article_id,price,sales_channel_id,count,user_id,item_id
0,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,663713001,0.050831,2,1,2,40179
1,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,541518023,0.030492,2,1,2,10520
2,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,505221004,0.015237,2,1,7,6387
3,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687003,0.016932,2,1,7,46304
4,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687004,0.016932,2,1,7,46305


In [6]:
tran = tran.drop(['t_dat','price', 'sales_channel_id'], axis=1)
tran.head(20)

,customer_id,article_id,count,user_id,item_id
0,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,663713001,1,2,40179
1,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,541518023,1,2,10520
2,00007d2de826758b65a93dd24ce629ed66842531df6699...,505221004,1,7,6387
3,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687003,1,7,46304
4,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687004,1,7,46305
5,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687001,1,7,46302
6,00007d2de826758b65a93dd24ce629ed66842531df6699...,505221001,1,7,6386
7,00083cda041544b2fbb0e0d2905ad17da7cf1007526fb4...,688873012,1,198,47416
8,00083cda041544b2fbb0e0d2905ad17da7cf1007526fb4...,501323011,1,198,5944
9,00083cda041544b2fbb0e0d2905ad17da7cf1007526fb4...,598859003,1,198,22540


In [7]:
rfm = pd.read_csv("data/result.csv")

In [21]:
rfm.head(3)

,customer_id,article_id,price,t_dat,age,count,total,t_dat_R,count_F,total_M,Class
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,176209023.0,0.035576,8467200.0,49.0,1,0.035576,2,1,8,2
1,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,568601006.0,0.050831,21340800.0,49.0,2,0.101661,4,9,10,1
2,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,568601043.0,0.050831,61862400.0,49.0,1,0.050831,10,1,9,1


In [8]:
rfm = rfm[['customer_id', 'article_id', 't_dat_R', 'count_F', 'total_M']]
sum_factor = (rfm['t_dat_R']*0.503095271342921) + (rfm['count_F']*0) + (rfm['total_M']*0.49690472865707896)
rfm['rfm'] = sum_factor

In [10]:
rfm = rfm.drop(['t_dat_R', 'count_F', 'total_M'], axis=1)

In [11]:
rfm.head()

,customer_id,article_id,rfm
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,176209023.0,4.981428
1,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,568601006.0,6.981428
2,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,568601043.0,9.503095
3,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,607642008.0,3.509286
4,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,625548001.0,5.478333


In [25]:
tran.head()

,customer_id,article_id,count,user_id,item_id
0,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,663713001,1,2,40179
1,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,541518023,1,2,10520
2,00007d2de826758b65a93dd24ce629ed66842531df6699...,505221004,1,7,6387
3,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687003,1,7,46304
4,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687004,1,7,46305


In [12]:
rfm = rfm.merge(tran, on=['customer_id', 'article_id'], how='right')

In [15]:
# rfm['customer_id'] = rfm['customer_id'].astype(int)
# rfm['article_id'] = rfm['article_id'].astype(int)
rfm['rfm'].max()

10.0

In [16]:
rfm.to_csv('data/df3.csv', index=False)

In [17]:
rfm.head()

,customer_id,article_id,rfm,count,user_id,item_id
0,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,663713001.0,5.472143,1,2,40179
1,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,541518023.0,3.484524,1,2,10520
2,00007d2de826758b65a93dd24ce629ed66842531df6699...,505221004.0,1.993809,1,7,6387
3,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687003.0,1.993809,1,7,46304
4,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687004.0,1.993809,1,7,46305


In [ ]:
# # coo 변환
# row = df['user_id'].values
# col = df['item_id'].values
# data = np.ones(tran.shape[0])
# coo_train = coo_matrix((data, (row, col)), shape=(len(ALL_USERS), len(ALL_ITEMS)))
# coo_train

In [18]:
df_spark = spark.read.csv("data/df3.csv", inferSchema=True, header=True)
# coo_train = spark.createDataFrame(coo_train)

In [19]:
train, test = df_spark.randomSplit([0.75, 0.25])

rec = ALS(maxIter=10,
        regParam=0.01,
        userCol='user_id',
        itemCol='article_id',
        ratingCol='rfm', # label -> predict할 때는 필요 없음!
        nonnegative=True,
        coldStartStrategy='drop')
# ALS모델 학습 -> dataframe을 넣어주기
rec_model = rec.fit(train)

# transform을 이용해 예측 -> dataframe을 넣어주기
pred_ratings = rec_model.transform(test)
pred_ratings.limit(5).toPandas()

22/05/02 19:48:27 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
22/05/02 19:48:27 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.ForeignLinkerBLAS


,customer_id,article_id,rfm,count,user_id,item_id,prediction
0,0001177027259b455f979d85a278e4b280205d4de5cce4...,811715001.0,8.993809,1,26,83291,8.491475
1,0001177027259b455f979d85a278e4b280205d4de5cce4...,851936003.0,7.503095,1,26,91960,8.522661
2,0001177027259b455f979d85a278e4b280205d4de5cce4...,852149001.0,7.503095,1,26,91998,8.899446
3,0001177027259b455f979d85a278e4b280205d4de5cce4...,777243002.0,6.509286,1,26,74572,8.991857
4,0001177027259b455f979d85a278e4b280205d4de5cce4...,810172003.0,8.993809,1,26,82842,9.370513


In [20]:
pred_ratings.limit(100).toPandas()

,customer_id,article_id,rfm,count,user_id,item_id,prediction
0,0001177027259b455f979d85a278e4b280205d4de5cce4...,811715001.0,8.993809,1,26,83291,8.491475
1,0001177027259b455f979d85a278e4b280205d4de5cce4...,851936003.0,7.503095,1,26,91960,8.522661
2,0001177027259b455f979d85a278e4b280205d4de5cce4...,852149001.0,7.503095,1,26,91998,8.899446
3,0001177027259b455f979d85a278e4b280205d4de5cce4...,777243002.0,6.509286,1,26,74572,8.991857
4,0001177027259b455f979d85a278e4b280205d4de5cce4...,810172003.0,8.993809,1,26,82842,9.370513
...,...,...,...,...,...,...,...
95,0008d644deb96bdc0ca262f161cf6d5e9a4e619bb75faa...,855896003.0,9.496905,1,210,92954,9.053293
96,0008d644deb96bdc0ca262f161cf6d5e9a4e619bb75faa...,855262002.0,9.006191,1,210,92822,8.702674
97,0008d644deb96bdc0ca262f161cf6d5e9a4e619bb75faa...,908729002.0,9.006191,1,210,103029,9.213879
98,000aa9a7d9f776173f87d58e834e4dd36f3a2d7908ae47...,625673004.0,3.006191,1,236,29577,4.125637


In [44]:
pre['prediction'].max()

0.97649825

In [21]:
# Get metric for training
from pyspark.ml.evaluation import RegressionEvaluator

evaluator = RegressionEvaluator(labelCol='rfm',
                              predictionCol='prediction',
                              metricName='rmse')
# evaluate 메소드에 예측값 담겨있는 dataframe 넣어주기
rmse = evaluator.evaluate(pred_ratings)

mae_eval = RegressionEvaluator(labelCol='rfm',
                              predictionCol='prediction',
                              metricName='mae')
mae = mae_eval.evaluate(pred_ratings)

print("RMSE:", rmse)
print("MAE:", mae)

RMSE: 1.0758949391149653
MAE: 0.7850179936343067


In [50]:
my_mc_lr = MulticlassClassificationEvaluator(predictionCol='prediction', labelCol='rfm', metricName='f1')
my_mc_lr.evaluate(pred_ratings)

IllegalArgumentException: requirement failed: Column prediction must be of type class org.apache.spark.sql.types.DoubleType$:double but was actually class org.apache.spark.sql.types.FloatType$:float.

In [22]:
# Pyspark Library #
# SQL
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark.sql.functions import mean, col, split, regexp_extract, when, lit
# ML
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, VectorAssembler, IndexToString
from pyspark.ml.feature import QuantileDiscretizer
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.recommendation import ALS


# stringIndexer = StringIndexer(inputCol='article_id',
#                             outputCol='article_new')
# print(stringIndexer)
# model = stringIndexer.bfit(df)
# indexed = model.transform(df)

# 숫자로 바꾼 영화제목들 중 Unique한 값들만 담아 추출하기 -> Dataframe 반환





In [56]:
pre1 = rfm
pre1.head(10)

,customer_id,article_id,rfm,count,user_id,item_id
0,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,663713001.0,20,1,2,40179
1,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,541518023.0,8,1,2,10520
2,00007d2de826758b65a93dd24ce629ed66842531df6699...,505221004.0,5,1,7,6387
3,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687003.0,5,1,7,46304
4,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687004.0,5,1,7,46305
5,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687001.0,5,1,7,46302
6,00007d2de826758b65a93dd24ce629ed66842531df6699...,505221001.0,6,1,7,6386
7,00083cda041544b2fbb0e0d2905ad17da7cf1007526fb4...,688873012.0,8,1,198,47416
8,00083cda041544b2fbb0e0d2905ad17da7cf1007526fb4...,501323011.0,11,1,198,5944
9,00083cda041544b2fbb0e0d2905ad17da7cf1007526fb4...,598859003.0,11,1,198,22540


In [25]:
df = spark.read.csv('data/df3.csv', inferSchema=True, header=True)
unique_movies = df.select("article_id").distinct()

def top_movies(user_id, n):
    """
    특정 user_id가 좋아할 만한 n개의 영화 추천해주는 함수
    """
    # unique_movies 데이터프레임을 'a'라는 데이터프레임으로 alias시키기
    a = unique_movies.alias('a')

    # 특정 user_id가 본 영화들만 담은 새로운 데이터프레임 생성
    watched_movies = df.filter(df['user_id'] == user_id)\
                            .select('article_id')

    # 특정 user_id가 본 영화들을 'b'라는 데이터프레임으로 alias시키기
    b = watched_movies.alias('b')

    # unique_movies를 기준으로 watched_movies를 조인시켜서 user_id가 보지 못한 영화들 파악 가능
    total_movies = a.join(b, a['article_id'] == b['article_id'],
                        how='left')

    # b 데이터프레임의 title_new값이 결측치를 갖고 있는 행의 a.title_new를 뽑아냄으로써 user_id가 아직 못본 영화들 추출
    # col('b.title_new') => b 데이터프레임의 title_new칼럼 의미(SQL처럼 가능!)
    remaining_movies = total_movies\
                    .where(col('b.article_id').isNull())\
                    .select('a.article_id').distinct()
    # remaining_movies 데이터프레임에 특정 user_id값을 동일하게 새로운 변수로 추가해주기
    remaining_movies = remaining_movies.withColumn('user_id',
                                                lit(int(user_id)))
    # 위에서 만든 ALS 모델을 사용하여 추천 평점 예측 후 n개 만큼 view -> 
    recommender = rec_model.transform(remaining_movies)\
                        .orderBy('prediction', ascending=False)\
                        .limit(n)

    # pre = recommender.toPandas()
    # pre1 = rfm.merge(pre, on=['user_id', 'article_id'], how='right')
    # pre1 = rfm.merge(pre)
    # return pre
    return recommender.show(n, truncate=False)

# userid가 1817번인 유저가 볼만한 영화 상위 5개 추천해주기
top_movies(26, 100)

# StringIndexer로 만든 것을 역으로 바꾸기 위해 IndexToString사용(영화제목을 숫자->한글제목)
    # movie_title = IndexToString(inputCol='article_new',
    #                         outputCol='article_id',
    #                         labels=model.labels) #여기서 model.labels는 StringIndexer에서 fit시켰을 때 생긴 레이블. 즉, 영화 제목들
    # # transform해서 영화제목을 숫자->한글로 변환! => dataframe으로 반환
    # final_recommendations = movie_title.transform(recommender)

+------------+-------+----------+
|article_id  |user_id|prediction|
+------------+-------+----------+
|8.42055001E8|26     |45.68711  |
|8.74928004E8|26     |43.714424 |
|8.65973002E8|26     |43.169563 |
|8.25437009E8|26     |42.730442 |
|8.57351001E8|26     |40.385006 |
|6.63406006E8|26     |37.172943 |
|8.20727005E8|26     |36.268536 |
|9.10439002E8|26     |35.85482  |
|8.04809001E8|26     |35.30089  |
|8.87552004E8|26     |33.605003 |
|9.03210004E8|26     |33.110954 |
|8.23724002E8|26     |32.769886 |
|7.25172001E8|26     |32.26112  |
|7.42019005E8|26     |31.079556 |
|4.97637024E8|26     |30.794016 |
|5.56959003E8|26     |30.543354 |
|9.10458001E8|26     |30.118803 |
|8.32157002E8|26     |29.685555 |
|8.84105003E8|26     |28.753965 |
|4.8524001E8 |26     |28.689281 |
|9.05416001E8|26     |28.540844 |
|7.54027003E8|26     |28.3501   |
|9.18642002E8|26     |28.34587  |
|9.14363003E8|26     |28.109035 |
|9.15851002E8|26     |27.96997  |
|9.21715001E8|26     |27.95468  |
|8.58306005E8|